<p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/7/74/Animation_for_a_deep_water_wave.gif" alt="Animation for a deep water wave" />
</p>

# Wave Height from Station 44013

---

## Overview
Generating a wavelet power spectrum from the time-series data [from wave heights from station 44013 East of Boston](https://www.ndbc.noaa.gov/station_history.php?station=44013)
1. Prerequisties
2. Background
3. Download and Organize Buoy Data
4. Wavelet Input Values
5. PyWavelets
6. Power Spectrum

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Matplotlib](https://foundations.projectpythia.org/core/matplotlib/matplotlib-basics.html) | Necessary | Used to plot data |
| [Intro to Pandas](https://foundations.projectpythia.org/core/pandas/pandas.html) | Necessary | Used to read in and organize data (in particular dataframes) |
| [Intro to Numpy](https://foundations.projectpythia.org/core/numpy/numpy-basics.html)| Necessary | Used to work with large arrays |

- **Time to learn**: 35 minutes

---

## Background

TODO

### Wavelets and Wave Height Data

TODO

## Imports

In [17]:
import numpy as np                          # working with arrays
import pandas as pd                         # working with dataframes
import matplotlib.pyplot as plt             # plot data
import datetime                             # converting strings to datetime objects
import requests                             # retrieve data from text on a website
import datetime

import pywt                                 # PyWavelets

## Access Buoy Data

We will be downloading the buoy data data from NOAA National Data Buoy Center with contains information about the buoy

The only columns we need to keep are related to datetimes (`YY`, `MM`, `DD`, `hh`) and wave height (`WVHT`)

In [18]:
years = [2001, 2002]
date_times = np.array([])
wave_height = np.array([])
for i, year in enumerate(years):
    data_link = f"https://www.ndbc.noaa.gov/view_text_file.php?filename=41001h{year}.txt.gz&dir=data/historical/stdmet/"
    data_req = requests.get(data_link)
    data_txt = data_req.text
    data = np.genfromtxt(data_txt.splitlines(), comments=None, dtype='str')
    datetime_index = np.where(data[0] == "hh")[0][0]
    wave_height_index = np.where(data[0] == "WVHT")[0][0]
    
    # colelct date data from full datset
    date_time = data[:,:datetime_index+1][1:]
    data_wave_height = data[:,wave_height_index][1:] # skip first header row
    data_wave_height = data_wave_height.astype(float)

    if i == 0:
        date_times = date_time
        wave_height = data_wave_height
    else:
        date_times = np.concatenate([date_times, date_time])
        wave_height = np.concatenate([wave_height, data_wave_height])

## Clean Up Wave Height data

Buoy stores null data as 99.00, so we will replace 99.00 with nan

In [19]:
print(max(wave_height))
wave_height = np.where(wave_height == 99.00, np.nan, wave_height)
print(max(wave_height))

99.0
9.38


## Convert Time to Datetimes

Buoy stores datetime information as seperate columns of data, let's convert this into a datetime object

In [20]:
# YYYY MM DD HH
dates = []
for date in date_times:
    date_string = "".join(date)
    dates.append(datetime.datetime.strptime(date_string, "%Y%m%d%H"))
dates = np.array(dates)
print(f"First Date = {dates[0]}")
print(f"Last Date = {dates[-1]}")

First Date = 2001-01-01 00:00:00
Last Date = 2002-12-17 20:00:00


## Plot and View Data

Let's give the data a look!

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title("Wave Height on Station 41001")
plt.xlabel("Date")
plt.xticks(rotation=90)
plt.ylabel("Wave Height (meters)")
plt.plot(dates, wave_height)
plt.show()

## Wavelet Input Values

Wavelet inputs include:

- x: Input time-series data (for example, the time and wave height of data)
- wavelet: mother wavelet name
- dt: sampling period (time between each y-value)
- s0: smallest scale
- dj: spacing between each discrete scales
- jtot: largest scale

In [ ]:
dt = 0.25  # sampling period (time between each y-value)
s0 = 0.25  # smallest scale
dj = 0.25  # spacing between each discrete scales
jtot = 64  # largest scale

**For this example, we will be using a complex Morlet with a bandwidth of 1.5 and a center frequency of 1**

In [ ]:
bandwidth = 1.5
center_freq = 1
wavelet_mother = f"cmor{bandwidth}-{center_freq}"
print(wavelet_mother)

## Applying Wavelets

Filter out nan values

In [ ]:
lm_len = np.arange(len(wave_height))
mask = np.isfinite(wave_height)
xfiltered = np.interp(lm_len, lm_len[mask], wave_height[mask])

In [ ]:
scales = np.arange(1, jtot + 1, dj)
wavelet_coeffs, freqs = pywt.cwt(
    data=xfiltered, scales=scales, wavelet=wavelet_mother, sampling_period=dt
)

f = pywt.scale2frequency(wavelet_mother, scales)/dt
#print(f)

## Power Spectrum
The power spectrum is the real component of the wavelet coefficients. We can find this value by squaring the absolute value of the `wavelet_coeffs` to return the magnitude of the real component to make a better graph.

In [ ]:
power = np.power((abs(wavelet_coeffs)), 2)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

# Plot Scalogram
plt.imshow(
    power, vmax=(power).max(), vmin=(power).min(), aspect="auto"
)

plt.title("Wave Height at Station 44013")
plt.xlabel("Date")
plt.ylabel("Scale")
#ax.set_xlim([dates[0], dates[-1]])
plt.colorbar()
plt.show()

The power spectrum above demonstrates a strong peak (in yellow) at  ...[TODO]

---

## Summary
TODO

### What's next?

- [Learn how more about PyWavelets](https://pywavelets.readthedocs.io/en/latest/regression/index.html)


## Resources and references

- ["A Practical Guide to Wavelet Analysis" (Torrence and Compo, 1998)](https://psl.noaa.gov/people/gilbert.p.compo/Torrence_compo1998.pdf)